Hello Fellas, this notebook is referred from best notebook so far for this particular competition https://www.kaggle.com/tanlikesmath/brain-tumor-radiogenomic-classification-eda-wip. I am no expert in this radiology field so I will avoid copy pasting amazing informative content written in referred notebook. Right now, I just tried to read dicom data using simple approach and plotting a 3D graph of it. I can say more such short but effective experiments are on the way in this notebook!

# Imports

At first, importing the required modules:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
import glob
from tqdm.notebook import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
import warnings
warnings.filterwarnings('ignore')

# A look at the provided data

Let's check what data is available to us:

In [ ]:
dataset_path = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification')

In [ ]:
train_df = pd.read_csv(dataset_path/'train_labels.csv')
print(f'There are {len(train_df)} patients in the dataset')

In [ ]:
train_df.head(10)

You might wonder what are MGMT values, as they mentioned in overview-
The presence of a specific genetic sequence in the tumor known as MGMT promoter methylation has been shown to be a favorable prognostic factor and a strong predictor of responsiveness to chemotherapy.

BraTS21ID-Its just an ID provided

Structure of the data:

* train_labels.csv - file containing the target `MGMT_value` for each of the 585 patients in the training data
* sample_submission.csv - a sample submission file for us to predict `MGMT_value`.
* train folder - comprises of MRI scans for 585 patients in DICOM format.
* test folder - The hidden test dataset is >5x the size of the training set.

In [ ]:
train_df['MGMT_value'].hist()

The distribution of the MGMT methylation status is fairly balanced.

# Dataset organization and MRI images

This competition dataset provides four types of MRI images:

1. [Fluid Attenuated Inversion Recovery (FLAIR)](https://en.wikipedia.org/wiki/Fluid-attenuated_inversion_recovery)
2. [T1-weighted pre-contrast (T1w)](https://radiopaedia.org/articles/t1-weighted-image?lang=us)
3. [T1-weighted post-contrast (T1Gd)](https://radiopaedia.org/articles/t1-weighted-image?lang=us)
4. [T2-weighted (T2)](https://radiopaedia.org/articles/t2-weighted-image?lang=us)


Picking 2 random scans as examples in order to visualize and 3D plot

In [ ]:
def load_dicoms(dcm_path):
    temp = []
    files = os.listdir(dcm_path)
    temp.append(files[0])
    temp.append(files[1])
    slices = [pydicom.dcmread(dcm_path + "/" + file) for file in temp]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))

    return slices

def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:
scans = load_dicoms('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00688/FLAIR')
print(scans)

In [ ]:
plt.imshow(scans[0].pixel_array, cmap="bone")

Rescaling few raw pixel values to H-Units

In [ ]:
def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

#     images = set_outside_scanner_to_air(images)
    
    # convert to HU
    for n in range(len(slices)):
        
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)
            
        images[n] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)

hu_scans = transform_to_hu(scans)

In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure 
def plot_3d(image, threshold=700, color="navy"):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces,_,_ = measure.marching_cubes_lewiner(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.2)
    mesh.set_facecolor(color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [ ]:
plot_3d(hu_scans)

Honestly, it doesn't make that much sense like when we 3D plot chest X-rays!